In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 4bit for 405b!
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    #model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit", # or choose "unsloth/Llama-3.2-1B-Instruct"
    model_name = "./email_extraction_Llama-3.2-3B-Instruct-bnb-4bit", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2024.10.7: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA GeForce RTX 4080 SUPER. Max memory: 15.695 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu124. CUDA = 8.9. CUDA Toolkit = 12.4.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.39it/s]
<string>:209: SyntaxWarning: invalid escape sequence '\ '
<string>:210: SyntaxWarning: invalid escape sequence '\_'
<string>:211: SyntaxWarning: invalid escape sequence '\ '
<string>:209: SyntaxWarning: invalid escape sequence '\ '
<string>:210: SyntaxWarning: invalid escape sequence '\_'
<string>:211: SyntaxWarning: invalid escape sequence '\ '
Unsloth: We fixed a gradient accumulation bug, but it seems like you don't have the latest transformers version!
Please update transformers, TRL and unsloth via:
`pip install --upgrade --no-cache-dir unsloth git+https://github.com/huggingface/transformers.git git+https://github.com/huggingface/trl.git`


In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)
model.print_trainable_parameters()

Unsloth 2024.10.7 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


trainable params: 24,313,856 || all params: 3,631,066,112 || trainable%: 0.6696


In [4]:
from datasets import load_dataset
dataset = load_dataset(
    "json",
    data_files = "./CargoOrderGoodStructure.json",
    split = "train",
)
print(dataset.column_names)
print(dataset[0])

['content', 'result']
{'content': "ently on hold within the\r\nMimecast service awaiting further action.\r\n\r\nFor further instructions on how to use the links associated with each email,\r\nplease review the following points:\r\n\r\nRelease: This will release the current email On Hold to your Inbox, but future\r\nemails from this sender will still be placed On Hold\r\nBlock: Rejects the email, and adds the sender's address to your personal Block\r\nlist to block future emails from this sender\r\nPermit: Delivers the email to your Inbox, and adds t", 'result': '{"cargoOrders":null}'}


In [5]:
print(dataset.column_names)

['content', 'result']


In [6]:
from unsloth import to_sharegpt
dataset = to_sharegpt(
    dataset,
    merged_prompt = \
        "[[{content}]]",
    #conversation_extension = 5, # Randomnly combines conversations into 1! Good for long convos
    output_column_name = "result",
)

# Replace 'gpt' with 'assistant' in the conversations
dataset = dataset.map(lambda x: {
    "conversations": [
        {"from": "user" if entry["from"] == "human" else "assistant", "value": entry["value"]}
        for entry in x["conversations"]
    ]
})

# Check the result
from pprint import pprint
pprint(dataset[0])

{'conversations': [{'from': 'user',
                    'value': '("ently on hold within the\\r\\nMimecast service '
                             'awaiting further action.\\r\\n\\r\\nFor further '
                             'instructions on how to use the links associated '
                             'with each email,\\r\\nplease review the '
                             'following points:\\r\\n\\r\\nRelease: This will '
                             'release the current email On Hold to your Inbox, '
                             'but future\\r\\nemails from this sender will '
                             'still be placed On Hold\\r\\nBlock: Rejects the '
                             "email, and adds the sender's address to your "
                             'personal Block\\r\\nlist to block future emails '
                             'from this sender\\r\\nPermit: Delivers the email '
                             'to your Inbox, and adds t",)'},
                   {'from': 'assi

In [7]:
from unsloth import standardize_sharegpt
dataset = standardize_sharegpt(dataset)

In [8]:
chat_template = """{SYSTEM}
USER: {INPUT}
ASSISTANT: {OUTPUT}"""

# Read the chat template from prompt.txt
with open("./EmailPrompt.txt", "r") as file:
    default_system_message = file.read()

from unsloth import apply_chat_template
dataset = apply_chat_template(
    dataset,
    tokenizer = tokenizer,
    chat_template = chat_template,
    default_system_message = default_system_message,
)

Unsloth: We automatically added an EOS token to stop endless generations.
Map: 100%|██████████| 18525/18525 [00:00<00:00, 51394.73 examples/s]


In [9]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size=8,  # Increase for stability if memory allows
        gradient_accumulation_steps=4,
        warmup_steps=50,  # Increase warmup for gradual ramp-up
        max_steps=200,  # More training steps
        learning_rate=1e-4,  # Lower learning rate for stability
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.1,  # Increased regularization
        lr_scheduler_type="cosine",  # Experiment with restarts
        seed=3407,
        output_dir="outputs",
    ),
)

Map (num_proc=2): 100%|██████████| 18525/18525 [00:12<00:00, 1452.68 examples/s]
max_steps is given, it will override any value given in num_train_epochs


In [10]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 4080 SUPER. Max memory = 15.695 GB.
2.674 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 18,525 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 8 | Gradient Accumulation steps = 4
\        /    Total batch size = 32 | Total steps = 200
 "-____-"     Number of trainable parameters = 24,313,856


**** Unsloth: Please use our fixed gradient_accumulation_steps by updating transformers, TRL and Unsloth!
`pip install --upgrade --no-cache-dir unsloth git+https://github.com/huggingface/transformers.git git+https://github.com/huggingface/trl.git`


Step,Training Loss


In [11]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

1509.7254 seconds used for training.
25.16 minutes used for training.
Peak reserved memory = 15.045 GB.
Peak reserved memory for training = 12.41 GB.
Peak reserved memory % of max memory = 95.859 %.
Peak reserved memory for training % of max memory = 79.07 %.


In [16]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
messages = [                    # Change below!
{"role": "user", "content": '''
hoffe alles gut :)\nWe can indic asf;\nBBC Rio or sub\nPOL: Bergen\nPOD: Shanghai, Cosco shipyard - Qidong\ncargo:\n1x hydraulic power unit: 13,56 x 4,20 x 2,87m / 44,43mt (as attached/below)\n1x hydraulic power unit: 13,56 x 4,20 x 2,87m / 44,04mt\n+ some spare parts (dets to follow)\nVia Suez\nfreight: USD 455.000 lpsm lthh\nTOS: 15-25 Feb\narrival until 06th April agw, wp, fme\nVia CogH\nfreight: USD 595.000 lpsm lthh\nTOS: 15-25 Feb\narrival until 09th April agw, wp, fme\npart cargo\nunder deck\npartly stackable\notherwise as per BBC BN\nsub further details\nsub costs/restrictions POD\nsub portcaptains approval of stowage
'''},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 3047, pad_token_id = tokenizer.eos_token_id)

 {"cargoOrders":[{"cargo_items":["1x hydraulic power unit: 13,56 x 4,20 x 2,87m / 44,43mt (as attached/below)","1x hydraulic power unit: 13,56 x 4,20 x 2,87m / 44,04mt","some spare parts (dets to follow)"],"loading_ports":["Bergen","Shanghai"],"discharging_ports":["Shanghai","Qidong"],"commission":0.0,"laycanPeriod":{"laycanFrom":"2024-02-01T00:00:00","laycanTo":"2024-02-25T00:00:00"}}]}<|eot_id|>


In [27]:
message_content = '''
Good day,\n\nPleased to hear with your indication for:\n\nAccount: DHL Global Forwarding IP\n\nCargo: 1 x motor - total about 179 cbm / about 86 mtons\n\nDimensions: 7,50 x 4,75 x 5,00 m – 86 tons\nAll basis +/- 10% in Charterer's option\nOtherwise as per drawings attached hereto\n\nCargo to be loaded under deck\nCargo to be non-stackable / non-overstowable\n\nLoadport: Hamburg or Bremen or Bremerhaven\n\nDischport: Darwin / Australia\n\nLaycan: 04th quarter 2024\n\nTerms: full liner terms hook/hook\nHooking on/off Merchant's account\nGrace period of 24hrs/24hrs\n\nTransittime: normal p/c\nPlease advise estimated transittime\n\nContract: BN\n\nCommission: 2.5% ttl on f/d/d/tf\n\nSubjects: Sub all further terms/conditions/details\n\nAwaiting yours in due course, many thanks for your continuous support.\n\nTake care,\nOttmar\n\nOttmar Schulte\nRegional Head of Marine Chartering Europe/Africa\nDGF - Industrial Projects\n\nDHL Global Forwarding GmbH\n\nOffice: +49 40 73355701\nCell: +49 15161578997
'''
messages = [              
    {"role": "user", "content": message_content }
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 3047, pad_token_id = tokenizer.eos_token_id)

 {"cargoOrders":[{"cargo_items":["1 x motor","179 cbm / 86 mtons"],"loading_ports":["Bremen","Hamburg","Bremerhaven"],"discharging_ports":["Darwin","Darwin Harbour"],"commission":2.5,"laycanPeriod":{"laycanFrom":"2024-10-01T00:00:00","laycanTo":"2024-10-31T00:00:00"}}]}<|eot_id|>


In [29]:
message_content = '''
Hope all is well?\n\nCould you please assist with rates for the following (TOS: 2nd half 2024 only)\n(Sorry for the many different options)\n\nOption A) Engines ex Vaasa\nPOL: Vaasa, FI\nCargo: 4511 cbm / 1454 tons / incl 3x 205 mt units\nPOD: \n1) Los Angeles\n2) Houston\n3) Basis 2 discharge ports (3 engine modules discharge in Houston, remaining cargo in Los Angeles)\n\nOption B) Step up Trafos\nPOL: Vaasa / Izmir\nPOD: Houston\nCargo: 971 cbm / 330 tons / incl 2x 105 mt units\nOption 1) Combination with above engines ex Vaasa\nOption 2) Separate shipment ex Vaasa\nOption 3) Separate shipment ex Turkey\n\nOption C) SCRs / mixing pipes\nCargo: 945 cbm / 77 tons\nPOL: Vaasa combination with above engines\n\nTerms: lthh\nUnder deck / deck option containers\nStackable as per PL\n\nThanks.
'''
messages = [              
    {"role": "user", "content": message_content }
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 3047, pad_token_id = tokenizer.eos_token_id)

 {"cargoOrders": [{"cargo_items": ["Engines ex Vaasa", "4511 cbm / 1454 tons / incl 3x 205 mt units"]}, "loading_ports": ["Vaasa"], "discharging_ports": ["Los Angeles", "Houston"], "commission": 0, "laycanPeriod": {"laycanFrom": "2024-06-01T00:00:00", "laycanTo": "2024-06-30T00:00:00"}}]}<|eot_id|>


In [30]:
message_content = '''
POL: Duqm, Oman\nPOD: Arzew, Algeria\nCommodity: Separation and Decarbonation Units\nQty: 9 units\nTotal weight: 1309 MT\nTotal volume: 5380.0332 cbm\nTerms: Requested Full Liner Terms (FLT) as part cargo\nPreference given to proposals with “Waiting for Berth” at both ends at Vessel Owner Account\nLaycan: 18th to 25th November 2024\nSee attached “Scope of Work” document for details\nStowage: Fully Under Deck, lashed and secured\nGrace period: 48 hours reversible POL/POD for loading/discharging\nStackability: Non-Stackable.
'''
messages = [              
    {"role": "user", "content": message_content }
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 3047, pad_token_id = tokenizer.eos_token_id)

 {"cargoOrders":[{"cargo_items":["Separation and Decarbonation Units","9 units"],"loading_ports":["Duqm"],"discharging_ports":["Arzew"],"commission":0.0,"laycanPeriod":{"laycanFrom":"2024-11-18T00:00:00","laycanTo":"2024-11-25T00:00:00"}}]}<|eot_id|>


In [32]:
message_content = '''
RFQ of 4 Transformadores desde Tanjung Priok Port – Jakarta, ---> DAP a Puerto rico // BBC CHARTERING
From lars.schoennemann@bbc-chartering.com
'''
messages = [              
    {"role": "user", "content": message_content }
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 3047, pad_token_id = tokenizer.eos_token_id)

 {"cargoOrders":null}<|eot_id|>


In [34]:
MODEL_NAME = 'email_extraction_Llama-3.2-3B-Instruct-bnb-4bit'

model.save_pretrained(MODEL_NAME) # Local saving
tokenizer.save_pretrained(MODEL_NAME)

('email_extraction_Llama-3.2-3B-Instruct-bnb-4bit/tokenizer_config.json',
 'email_extraction_Llama-3.2-3B-Instruct-bnb-4bit/special_tokens_map.json',
 'email_extraction_Llama-3.2-3B-Instruct-bnb-4bit/tokenizer.json')

In [35]:
model.save_pretrained_gguf(MODEL_NAME, tokenizer,)

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### Your chat template has a BOS token. We shall remove it temporarily.


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 11.95 out of 30.47 RAM for saving.


100%|██████████| 28/28 [00:00<00:00, 98.88it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp will take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits will take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q8_0'] will take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: [0] Installing llama.cpp. This will take 3 minutes...
Unsloth: [1] Converting model at email_extraction_Llama-3.2-3B-Instruct-bnb-4bit into q8_0 GGUF format.
The output location will be /home/jovyan/work/email_extraction_Llama-3.2-3B-Instruct-bnb-4bit/unsloth.Q8_0.gguf
This will take 3 minutes...
INFO:hf-to-gguf:Loading model: email_extraction_Llama-3.2-3B-Instruct-bnb-4bit
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {64}
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### We removed it in GGUF's chat template for you.


Unsloth: Conversion completed! Output location: /home/jovyan/work/email_extraction_Llama-3.2-3B-Instruct-bnb-4bit/unsloth.Q8_0.gguf
Unsloth: Saved Ollama Modelfile to email_extraction_Llama-3.2-3B-Instruct-bnb-4bit/Modelfile
